In [1]:
import socket

NB='test'
DESCRIPTION='test notebook'
HOST = socket.gethostname()
HOST

'DESKTOP-7KHT10A'

In [2]:
import warnings
import os
from pathlib import Path

import mlflow
from dataclasses import dataclass
from dataclasses import asdict

import pandas as pd
#import lightgbm as lgb
import xgboost as xgb
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, mean_squared_error, roc_auc_score, average_precision_score, mean_squared_log_error
from sklearn.preprocessing import LabelEncoder

#from my_utils import get_notebook_path

warnings.simplefilter('ignore')
pd.set_option('display.max_columns', 25)
pd.set_option('display.max_rows', 25)

In [3]:
ROOT_DIR = Path('../')
DATA_DIR = ROOT_DIR / Path('data')

In [4]:
class Config:
    N_LABEL = 24
    N_FOLD = 5
    RANDOM_SATE = 42
    LR = 0.15
    PATIENCE = 30
    EPOCH = 300
    BATCH_SIZE = 6
    SKIP_EVALUATE_NUM = 5

In [5]:
train_df = pd.read_csv(DATA_DIR / Path('titanic_train.csv'))
test_df = pd.read_csv(DATA_DIR / Path('titanic_test.csv'))

display(train_df.shape)
display(train_df.head(5))
display(test_df.shape)
display(test_df.head(5))

(891, 12)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


(418, 11)

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


## feature and target

In [6]:
target = 'Survived'
del_columns = ['Survived', 'Name']

features = list(set(train_df.columns) - set(del_columns))
features

['Pclass',
 'Embarked',
 'Sex',
 'Fare',
 'Age',
 'SibSp',
 'Cabin',
 'Ticket',
 'Parch',
 'PassengerId']

## train lgbm

### metrix

In [7]:
def evaluation(true, pred):
    return roc_auc_score(true, pred)

### encording

In [8]:
# 数値でないカラムをLabelEncording
categoricals = ['Sex', 'Ticket', 'Cabin','Embarked']
features = list(set(features) | set(categoricals))
for c in categoricals:
    train_df[c] = train_df[c].astype("category")
    test_df[c] = test_df[c].astype("category")

# Count encoding
count_enc = ['Sex', 'Ticket', 'Cabin', 'Embarked']
count_features = []
for c in count_enc:
    _count = train_df.append(test_df)[c].astype(str).value_counts().to_dict()
    train_df[f'{c}_count'] = train_df[c].astype(str).map(_count)
    test_df[f'{c}_count'] = test_df[c].astype(str).map(_count)
    count_features.append(f'{c}_count')
features = list(set(features) | set(count_features))

### train

In [9]:
params = {
    'objective': 'reg:squarederror',
    'random_state': Config.RANDOM_SATE, 
    'eval_metric': 'auc',
    'learning_rate': 0.001
}

# TargetEncording Config
target_enc = ['Sex', 'Ticket', 'Cabin', 'Embarked'] # 集計対象カラム
target_enc_key = ['Survived'] # 集計するキー

oof_preds = np.zeros(len(train_df))
y_pred = np.zeros(len(test_df))
models = []
cv_scores = {}
skf = StratifiedKFold(n_splits=Config.N_FOLD, random_state=Config.RANDOM_SATE, shuffle=True)
for fold, (train_index, test_index) in enumerate(skf.split(train_df[features], train_df[target])):

    print(f'====== fold {fold} ======')

    # TrainとTestに分割
    x_train, x_val = train_df.copy().iloc[train_index][features], train_df.copy().iloc[test_index][features]
    y_train, y_val =  train_df.iloc[train_index][target], train_df.iloc[test_index][target]

    test = test_df[features]

    # Target Encoding
    if len(target_enc) > 0:
        for t in target_enc_key:
            for c in target_enc:
                x_train[f'{c}_target_enc_by_{t}'] = train_df.iloc[train_index][c].map(train_df.iloc[test_index].groupby(c)[t].mean().to_dict())
                x_val[f'{c}_target_enc_by_{t}'] = train_df.iloc[test_index][c].map(train_df.groupby(c)[t].mean().to_dict())
                test[f'{c}_target_enc_by_{t}'] = test_df[c].map(train_df.groupby(c)[t].mean().to_dict())
            
    train_features = x_train.columns.to_list()

    # create Dataset
    dtrain = xgb.DMatrix(x_train, label=y_train, enable_categorical=True)
    dvalid = xgb.DMatrix(x_val, label=y_val, enable_categorical=True)
    dtest = xgb.DMatrix(test[x_train.columns], enable_categorical=True)

    # train
    model = xgb.train(params, dtrain, 5000, early_stopping_rounds=100, evals=[(dtrain, 'train'), (dvalid, 'eval')], verbose_eval=100)
    models.append(model)

    fold_pred = model.predict(dvalid, ntree_limit = model.best_ntree_limit)
    
    score = evaluation(y_val, fold_pred)
    cv_scores[f'cv{fold}'] = score

    oof_preds[test_index] = fold_pred

    y_pred += model.predict(dtest, ntree_limit = model.best_ntree_limit) / Config.N_FOLD

    print(f'cv score is {score}')

oof_score = evaluation(train_df[target], oof_preds)
print(f'OOF score is {oof_score}')

====== fold 0 ======
[0]	train-auc:0.92380	eval-auc:0.89618
[99]	train-auc:0.92947	eval-auc:0.89025
cv score is 0.896179183135705
====== fold 1 ======
[0]	train-auc:0.92550	eval-auc:0.83322
[100]	train-auc:0.92556	eval-auc:0.84612
[105]	train-auc:0.92556	eval-auc:0.84612
cv score is 0.8474598930481282
====== fold 2 ======
[0]	train-auc:0.92816	eval-auc:0.80662
[100]	train-auc:0.93779	eval-auc:0.80735
[192]	train-auc:0.93866	eval-auc:0.80722
cv score is 0.8082887700534759
====== fold 3 ======
[0]	train-auc:0.92062	eval-auc:0.92567
[99]	train-auc:0.92380	eval-auc:0.91270
cv score is 0.925668449197861
====== fold 4 ======
[0]	train-auc:0.93920	eval-auc:0.85521
[100]	train-auc:0.93975	eval-auc:0.85939
[139]	train-auc:0.93942	eval-auc:0.86019
cv score is 0.8621858795372956
OOF score is 0.8272936439459304
